#### 用keras的vgg模型实现在cifar10的迁移学习


-  参考:
    > [keras Application](https://keras-cn.readthedocs.io/en/latest/other/application/)  
    > [使用 Google Inception V3模型进行迁移学习](https://imyong.top/2018/05/30/Inception-V3-implementation-17-flowers-classes/#4.%E7%94%A8Google-Inception-V3-%E6%A8%A1%E5%9E%8B%E5%81%9A%E8%BF%81%E7%A7%BB%E5%AD%A6%E4%B9%A0)  
    > [kaggle](https://www.kaggle.com/xhlulu/vgg-16-on-cifar10-with-keras-beginner) 
    

#### 1.处理数据

In [ ]:
import os
import json
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.applications.vgg16 import preprocess_input
from keras.utils import to_categorical


def unpickle(file, encoding='bytes'):
    with open(file, 'rb') as f:
        di = pickle.load(f, encoding=encoding)
    return di


batch_labels = []
batch_images = []

for n in range(1, 6):
    batch_dict = unpickle(f"/input0/cifar-10-batches-py/data_batch_{n}")
    # Add labels to the list of batch labels
    batch_labels.append(batch_dict[b'labels'])

    # Load the images, and resize them to 10000x3x32x32
    data = batch_dict[b'data'].reshape((10000, 3, 32, 32))
    # Modify axis to be 10000x32x32x3, since this is the correct order for keras
    data = np.moveaxis(data, 1, -1)
    batch_images.append(data)

labels = np.concatenate(batch_labels, axis=0)
images = np.concatenate(batch_images, axis=0)

test_dict = unpickle(f"/input0/cifar-10-batches-py/test_batch")
test_labels = np.array(test_dict[b'labels'])
test_images = test_dict[b'data'].reshape((10000,3,32,32))
test_images = np.moveaxis(test_images, 1, -1)


# We normalize the input according to the methods used in the paper
X_train = preprocess_input(images)
y_train = to_categorical(labels)

# We one-hot-encode the labels for training
X_test = preprocess_input(test_images)
y_test = to_categorical(test_labels)


#### 2.模型
> 提取出了vgg16的卷积层，增加了两个全连接层，在训练的时候vgg16的卷积层参数不参与训练

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import numpy as np

base_model = VGG16(weights='imagenet', include_top=False)

# base_model.summary()

x=base_model.output
x = GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x)
predictions=Dense(10,activation='softmax')(x)


model=Model(inputs=base_model.input,outputs=predictions)
for layer in base_model.layers:
    layer.trainable=False
model.summary()

model.compile(optimizer='sgd',loss='categorical_crossentropy')
# a=model.predict(np.array([X_train[0]]))
# print(a)
# print(a.shape)
# print(y_train[0].shape)
checkpoint = ModelCheckpoint(
    'model.h5',
    monitor='val_acc',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto'
)
history=model.fit(X_train,y=y_train,validation_split=0.1,batch_size=200,epochs=10,callbacks=[checkpoint])
model.evaluate(x=X_test,y=y_test)

